### Initialize the dataset

In [ ]:
import phanotate_modules.functions as phano
import numpy as np
import pyfastx
import phanotate_modules.functions as phano
shotsetds=[]
shotsetss=[]

fa_ds = pyfastx.Fasta('ds-linear.fasta')
fa_ss = pyfastx.Fasta('ss-circular.fasta')
fa_ds_1 = pyfastx.Fasta('ds-circular.fasta')
fa_ss_1 = pyfastx.Fasta('ss-linear.fasta')
def shot(str):
    length=len(str)
    start=np.random.randint(0,length,10)
    shot_len=np.random.normal(3000,200,1)
    return [str[s:int(s+shot_len)] for s in start]

for itm in fa_ds:
    for _ in range(1):
        shotsetds=shotsetds+shot(itm.seq)

for itm in fa_ds_1:
    for _ in range(1):
        shotsetds=shotsetds+shot(itm.seq)

for itm in fa_ss_1:
    for _ in range(1):
        shotsetss=shotsetss+shot(itm.seq)

for itm in fa_ss:
    for _ in range(1):
        shotsetss=shotsetss+shot(itm.seq)
shotsetds=[phano.get_backgroud_rbs(i) for i in shotsetds]
# shotsetds = torch.Tensor(shotsetds)
# shotsetds = torch.Tensor.reshape(shotsetds,(-1,28))
shotsetss=[phano.get_backgroud_rbs(i) for i in shotsetss]
# shotsetss = torch.Tensor(shotsetss)
# shotsetss = torch.Tensor.reshape(shotsetss,(-1,28))
shotsetlist=shotsetds+shotsetss
shotsetlist=np.array(shotsetlist,dtype=np.float32)
labels1=np.zeros(len(fa_ds)*10)
labels2=np.zeros(len(fa_ds_1)*10)+1
labels3=np.zeros(len(fa_ss_1)*10)+2
labels4=np.zeros(len(fa_ss)*10)+3
# label=[labels1,labels2,labels3,labels4]
label=np.r_[labels1,labels2,labels3,labels4]
label=np.array(label,dtype=np.int)

filename="result.txt"
filename= save_variable(shotsetlist,filename)

### Two-Class label

In [ ]:
shotsetlist=load_variavle("result.txt")
labels1=np.zeros(len(fa_ds)*10+len(fa_ds_1)*10)
labels2=np.zeros(len(fa_ss_1)*10+len(fa_ss)*10)+1
label=np.r_[labels1,labels2]
label=np.array(label,dtype=np.int)

In [ ]:
labels1=np.zeros(len(fa_ds)*10)
labels2=np.zeros(len(fa_ds_1)*10)+1
labels3=np.zeros(len(fa_ss_1)*10)+2
labels4=np.zeros(len(fa_ss)*10)+3
label=np.r_[labels1,labels2,labels3,labels4]
label=np.array(label,dtype=np.int)

In [ ]:

import pickle
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
def load_variavle(filename):
   f=open(filename,'rb')
   r=pickle.load(f)
   f.close()
   return r

shotsetlist=load_variavle("result.txt")
labels1=np.zeros(len(fa_ds)*10+len(fa_ds_1)*10)
# labels2=np.zeros(len(fa_ds_1)*10)+1
labels2=np.zeros(len(fa_ss_1)*10+len(fa_ss)*10)+1
# labels4=np.zeros(len(fa_ss)*10)+3
# label=[labels1,labels2,labels3,labels4]
label=np.r_[labels1,labels2]
label=np.array(label,dtype=np.int)

### Initialize the classifier

In [ ]:
from sklearn.model_selection import train_test_split
#x为数据集的feature熟悉，y为label.
x_train, x_test, y_train, y_test = train_test_split(shotsetlist[0:len(labels1)+len(labels2)], label[0:len(labels1)+len(labels2)], test_size = 0.2,random_state=4)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(shotsetlist)
shotsetlist=scaler.transform(shotsetlist)

## DNN method 

### Initialize the Network

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from d2l import torch as d2l

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)


# Hyper-parameters
input_size = 1
output_size = 1
num_epochs = 100
model = nn.Sequential(nn.Linear(28,16),nn.Linear(16,128),nn.ReLU(),nn.Linear(128,2))
model.apply(init_weights)

# Loss and optimizer
# weight=torch.from_numpy(np.array([0.1,1.0,1.0,1.0],dtype=np.float32))
loss = nn.CrossEntropyLoss(size_average=True)
trainer = torch.optim.SGD(model.parameters(), lr=1)  


# Train the model
# d2l.train_ch3(model, x_train, y_train, loss, num_epochs, trainer)
from torch.utils.data import DataLoader
from torch.utils import data
class CustomDataset(data.Dataset):#需要继承data.Dataset
    def __init__(self,x,y):
        # TODO
        # 1. Initialize file path or list of file names.
        self.X=x 
        self.y=y
        self.length=len(self.X)
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        return self.X[index],self.y[index]
        
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return self.length

### Training and prediction

In [ ]:
shotset=CustomDataset(torch.Tensor(x_train),y_train)
myloader=data.DataLoader(shotset,batch_size=256)
testset=CustomDataset(torch.Tensor(x_test),y_test)
testloader=data.DataLoader(shotset,batch_size=32 )
d2l.train_ch3(model,myloader,testloader,loss,100,trainer)
trainer = torch.optim.SGD(model.parameters(), lr=0.1)  
d2l.train_ch3(model,myloader,testloader,loss,100,trainer)
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition  # 这里应用了广播机制
res=softmax(model(torch.Tensor(x_test)))
res=softmax(model(torch.Tensor(x_test)))
filtered_idx=[]
number=0
right_num=0
for i in range(len(res)):
    idx=res[i].argmax(0)
    if(res[i][idx]>0.71):
        filtered_idx.append(i)
        number=number+1
        if (idx== y_test[i]):
            right_num=right_num+1
right_num/number , number/len(y_test)
train(model,myloader,test_iter=testloader,loss=loss,num_epochs=1000,trainer=trainer)

### Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
#x为数据集的feature熟悉，y为label.
# x_train, x_test, y_train, y_test = train_test_split(dna_data, labels, test_size = 0.2,random_state=4)
from sklearn import tree
X = x_train
Y = y_train
clf = tree.DecisionTreeClassifier()
clf.max_depth=3
clf = clf.fit(X, Y)
# scores=cross_val_score(clf,x_test,y_test)
# scores.mean()

### High Precision Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(max_depth=None, min_samples_leaf=2,random_state=0,bootstrap = True,
n_estimators=200,oob_score=True)
clf = clf.fit(x_train,y_train)


number=0
right_num=0
index=[]
for i in range(len(x_test)):
    result=clf.predict_proba(x_test[i:i+1])
    pred=clf.predict(x_test[i:i+1])
    if np.max(result)>0.45:
        number=number+1
        index.append(i)
        if pred[0]==y_test[i]:
            right_num=right_num+1
        
right_num/number

### Ada Boost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
clf = AdaBoostClassifier(n_estimators=150,base_estimator=tree.DecisionTreeClassifier())
clf = clf.fit(x_train,y_train)
clf.score(x_test,y_test)

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(x_train,y_train)
scores=cross_val_score(gnb,x_test,y_test)
scores.mean()


## Model Analysis
### AUC score and map for 2-class

In [ ]:
from sklearn.metrics import roc_curve,auc,roc_auc_score
fpr, tpr, thresholds= roc_curve(y_test,clf.predict(x_test))
from matplotlib import pyplot as plt
auc(fpr, tpr)
roc_auc_score(y_test,clf.predict(x_test))

### Confusion Matrix

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import pandas as pd
index=[i for i in range(len(x_test)) if i in filtered_idx ]
y_true=y_train
# y_pred=clf.predict(x_train)
C=confusion_matrix(y_test[index], model(torch.Tensor(x_test[index])).argmax(1))
index=["ds-linear", "ds-circular","ss-linear","ss-circular"]
df=pd.DataFrame(C,index=index[0:2],
columns=index[0:2])
sns.heatmap(df,annot=True)